In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
# MPC import
import numpy as np
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:

# 1. Setup Parameters
Ts = 0.05           # Sampling time (Fixed)
sim_time = 10.0     # Simulation time (7s is the requirement, 10s is safe to view stability)
H = 2.0             # Prediction Horizon in Seconds

# 2. Define Initial State (Stationary at 40 degrees roll)
# State vector: [wx, wy, wz, alpha, beta, gamma, vx, vy, vz, x, y, z]
x0 = np.zeros(12)
x0[5] = np.deg2rad(40) # Gamma (Roll) = 40 deg
# x0[6:9] = np.array([5.0] *3)  # Initial Velocity (vx, vy, vz)

# 3. Initialize Rocket and Controller
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, method="linear"
)

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol);